In [0]:
%run ../config

In [0]:
%run ./00-global-setup-v2

In [0]:
reset_all_data = dbutils.widgets.get("reset_all_data") == "true"
DBDemos.setup_schema(catalog, db, reset_all_data, volume_name)
folder = f"/Volumes/{catalog}/{db}/{volume_name}"

In [0]:
def sensor_hourly_agg(input_table: str):
    df = spark.table(input_table)
    result = (
        df.groupBy(
            "turbine_id",
            F.date_trunc("hour", F.from_unixtime("timestamp")).alias("hourly_timestamp")
        )
        .agg(
            F.avg("energy").alias("avg_energy"),
            F.stddev_pop("sensor_A").alias("std_sensor_A"),
            F.stddev_pop("sensor_B").alias("std_sensor_B"),
            F.stddev_pop("sensor_C").alias("std_sensor_C"),
            F.stddev_pop("sensor_D").alias("std_sensor_D"),
            F.stddev_pop("sensor_E").alias("std_sensor_E"),
            F.stddev_pop("sensor_F").alias("std_sensor_F"),
            F.expr("percentile_approx(sensor_A, array(0.1,0.3,0.6,0.8,0.95))").alias("percentiles_sensor_A"),
            F.expr("percentile_approx(sensor_B, array(0.1,0.3,0.6,0.8,0.95))").alias("percentiles_sensor_B"),
            F.expr("percentile_approx(sensor_C, array(0.1,0.3,0.6,0.8,0.95))").alias("percentiles_sensor_C"),
            F.expr("percentile_approx(sensor_D, array(0.1,0.3,0.6,0.8,0.95))").alias("percentiles_sensor_D"),
            F.expr("percentile_approx(sensor_E, array(0.1,0.3,0.6,0.8,0.95))").alias("percentiles_sensor_E"),
            F.expr("percentile_approx(sensor_F, array(0.1,0.3,0.6,0.8,0.95))").alias("percentiles_sensor_F"),
        )
    )
    display(result)

In [0]:
%sql
CREATE OR REPLACE FUNCTION sensor_hourly_agg(input_table STRING)
RETURNS TABLE (
  turbine_id STRING,
  hourly_timestamp TIMESTAMP,
  avg_energy DOUBLE,
  std_sensor_A DOUBLE,
  std_sensor_B DOUBLE,
  std_sensor_C DOUBLE,
  std_sensor_D DOUBLE,
  std_sensor_E DOUBLE,
  std_sensor_F DOUBLE,
  percentiles_sensor_A ARRAY<DOUBLE>,
  percentiles_sensor_B ARRAY<DOUBLE>,
  percentiles_sensor_C ARRAY<DOUBLE>,
  percentiles_sensor_D ARRAY<DOUBLE>,
  percentiles_sensor_E ARRAY<DOUBLE>,
  percentiles_sensor_F ARRAY<DOUBLE>
)
RETURN
SELECT
  turbine_id,
  date_trunc('hour', from_unixtime(timestamp)) AS hourly_timestamp,
  avg(energy) AS avg_energy,
  stddev_pop(sensor_A) AS std_sensor_A,
  stddev_pop(sensor_B) AS std_sensor_B,
  stddev_pop(sensor_C) AS std_sensor_C,
  stddev_pop(sensor_D) AS std_sensor_D,
  stddev_pop(sensor_E) AS std_sensor_E,
  stddev_pop(sensor_F) AS std_sensor_F,
  percentile_approx(sensor_A, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_A,
  percentile_approx(sensor_B, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_B,
  percentile_approx(sensor_C, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_C,
  percentile_approx(sensor_D, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_D,
  percentile_approx(sensor_E, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_E,
  percentile_approx(sensor_F, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_F
FROM TABLE(IDENTIFIER(input_table))
GROUP BY
  turbine_id,
  date_trunc('hour', from_unixtime(timestamp));

In [0]:
%sql
CREATE OR REPLACE VIEW sensor_hourly_agg_live AS
SELECT
  turbine_id,
  date_trunc('hour', from_unixtime(timestamp)) AS hourly_timestamp,
  avg(energy) AS avg_energy,
  stddev_pop(sensor_A) AS std_sensor_A,
  stddev_pop(sensor_B) AS std_sensor_B,
  stddev_pop(sensor_C) AS std_sensor_C,
  stddev_pop(sensor_D) AS std_sensor_D,
  stddev_pop(sensor_E) AS std_sensor_E,
  stddev_pop(sensor_F) AS std_sensor_F,
  percentile_approx(sensor_A, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_A,
  percentile_approx(sensor_B, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_B,
  percentile_approx(sensor_C, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_C,
  percentile_approx(sensor_D, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_D,
  percentile_approx(sensor_E, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_E,
  percentile_approx(sensor_F, array(0.1, 0.3, 0.6, 0.8, 0.95)) AS percentiles_sensor_F
FROM LIVE.sensor_bronze
GROUP BY
  turbine_id,
  date_trunc('hour', from_unixtime(timestamp));

In [0]:
%sql
SELECT * FROM public_sector.predictive_maintenance_navy_test.historical_sensor_data LIMIT(10)

In [0]:
# -- Refactored to PySpark (for reference, not executable in %sql cell)
# -- Please copy this code into a Python cell

from pyspark.sql import functions as F

def sensor_hourly_agg(df):
    return (
        df.withColumn("hourly_timestamp", F.date_trunc("hour", F.from_unixtime(F.col("timestamp"))))
          .groupBy("turbine_id", "hourly_timestamp")
          .agg(
              F.avg("energy").alias("avg_energy"),
              F.stddev_pop("sensor_A").alias("std_sensor_A"),
              F.stddev_pop("sensor_B").alias("std_sensor_B"),
              F.stddev_pop("sensor_C").alias("std_sensor_C"),
              F.stddev_pop("sensor_D").alias("std_sensor_D"),
              F.stddev_pop("sensor_E").alias("std_sensor_E"),
              F.stddev_pop("sensor_F").alias("std_sensor_F"),
              F.expr("percentile_approx(sensor_A, array(0.1, 0.3, 0.6, 0.8, 0.95))").alias("percentiles_sensor_A"),
              F.expr("percentile_approx(sensor_B, array(0.1, 0.3, 0.6, 0.8, 0.95))").alias("percentiles_sensor_B"),
              F.expr("percentile_approx(sensor_C, array(0.1, 0.3, 0.6, 0.8, 0.95))").alias("percentiles_sensor_C"),
              F.expr("percentile_approx(sensor_D, array(0.1, 0.3, 0.6, 0.8, 0.95))").alias("percentiles_sensor_D"),
              F.expr("percentile_approx(sensor_E, array(0.1, 0.3, 0.6, 0.8, 0.95))").alias("percentiles_sensor_E"),
              F.expr("percentile_approx(sensor_F, array(0.1, 0.3, 0.6, 0.8, 0.95))").alias("percentiles_sensor_F"),
          )
    )